In [17]:
import pandas as pd
import re
import datetime


# =========== Pre-process ==========
df = pd.read_csv('/Users/linzeyang/Desktop/BIA-660/201606-citibike-tripdata.csv')

# Missing values in columns
print(df.isnull().sum())


# Snake_case the columns
def camel_to_snake(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).replace(' ','').lower()
df.columns = [camel_to_snake(col) for col in df.columns]

# Parse start_time and stop_time
df['start_datetime'] = [datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in df.starttime]
df['start_day'] = [x.strftime('%m/%d/%Y') for x in df.start_datetime]
# The other way
# df['start_day'] = [x[:10] for x in df.start_time]
# print(df.head())

# =========== Group by date and station ===========
df['bikein'] = 1
df['bikeout'] = 1

df_bikeout = df.groupby(by=['startstationid', 'start_day'])['bikeout'].count()
df_bikeout = pd.DataFrame(df_bikeout)
df_out = df_bikeout.reset_index(drop = False)
df_out.rename_axis({'startstationid':'station_id'}, axis='columns', inplace=True)
print(df_out.head())

df_bikein = df.groupby(by=['endstationid', 'start_day'])['bikein'].count()
df_bikein = pd.DataFrame(df_bikein)
df_in = df_bikein.reset_index(drop = False)
df_in.rename_axis({'endstationid':'station_id'}, axis='columns', inplace=True)
print(df_in.head())


# =========== Bike loss per day for each station ===========
# Merge two DFs by station_id
df_dayloss = df_out.merge(right=df_in, left_on=['station_id', 'start_day'], right_on=['station_id', 'start_day'],
                          how='outer')
df_dayloss = df_dayloss.sort_values(['station_id', 'start_day'])
df_dayloss = df_dayloss.fillna(0)
df_dayloss['bike_loss'] = df_dayloss['bikeout'] - df_dayloss['bikein']
print(df_dayloss)

tripduration                    0
starttime                       0
stoptime                        0
start station id                0
start station name              0
start station latitude          0
start station longitude         0
end station id                  0
end station name                0
end station latitude            0
end station longitude           0
bikeid                          0
usertype                        0
birth year                 195206
gender                          0
dtype: int64
   station_id   start_day  bikeout
0          72  06/01/2016      143
1          72  06/02/2016      135
2          72  06/03/2016       90
3          72  06/04/2016      123
4          72  06/05/2016       55
   station_id   start_day  bikein
0          72  06/01/2016     144
1          72  06/02/2016     131
2          72  06/03/2016      91
3          72  06/04/2016     126
4          72  06/05/2016      50
       station_id   start_day  bikeout  bikein  bike_loss
0    

In [69]:
# create weekday column
#wekday_name=pd.to_datetime(df_dayloss.start_day).dt.weekday_name   #name
wekday_index=pd.to_datetime(df_dayloss.start_day).dt.weekday   # index Monday=0, Sunday=6

In [76]:
# Append weekday column to df_dayloss
df_dayloss['wek_index']=df_wekday_index

In [77]:
df_dayloss

,station_id,start_day,bikeout,bikein,bike_loss,wek_index
0,72,06/01/2016,143.0,144.0,-1.0,2
1,72,06/02/2016,135.0,131.0,4.0,3
2,72,06/03/2016,90.0,91.0,-1.0,4
3,72,06/04/2016,123.0,126.0,-3.0,5
4,72,06/05/2016,55.0,50.0,5.0,6
5,72,06/06/2016,126.0,127.0,-1.0,0
6,72,06/07/2016,128.0,126.0,2.0,1
7,72,06/08/2016,83.0,94.0,-11.0,2
8,72,06/09/2016,122.0,119.0,3.0,3
9,72,06/10/2016,158.0,145.0,13.0,4
